In [1]:
# Retreiver
# Use appropriate tokenizer and prepare data for inference

# Ranker
# Use appropriate tokenizer and prepare data for inference

In [2]:
import sys
import os
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = os.environ.get('KAGGLE_KERNEL_RUN_TYPE','') # got this from https://www.kaggle.com/code/jamesmcguigan/kaggle-environment-variables-os-environ/notebook

if IN_COLAB:

    # Enable line-wrapping
    # from IPython.display import HTML, display

    # def set_css():
    #     display(HTML('''
    #         <style>
    #         pre {
    #             white-space: pre-wrap;
    #             }
    #         </style>
    #     '''))
    # get_ipython().events.register('pre_run_cell', set_css)


    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

    %pip install transformers
    %pip install datasets
    %pip install pynvml

In [3]:
print(IN_KAGGLE)
print(IN_COLAB)


False


In [4]:
import datetime
import numpy as np
from sklearn.metrics import fbeta_score
from matplotlib import pyplot as plt
import math
import pandas as pd
import json
import time
import torch
from torch import nn
import random
#from tqdm.auto import tqdm
from tqdm import tqdm
from torch.utils.data import DataLoader
from itertools import product

from transformers import AutoTokenizer, AutoModel
from dataclasses import dataclass
from transformers import PreTrainedTokenizer
from torch.utils.data import DataLoader

if IN_COLAB:
    sys.path.append('/content/drive/MyDrive/ML/lecr/code/')
elif IN_KAGGLE:
    sys.path.append('/kaggle/input/lecr-utils/')
from l_utils import *
from model_utils import *
from fe_utils import *

#from model_defs import Topic_NN, Content_NN

%env CUBLAS_WORKSPACE_CONFIG=:16:8

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

/home/mahesh/Desktop/ML/kaggle/LearningEquality/learning-equality-code/le-venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-17 14:08:33.517019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 14:08:33.649382: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2023-04-17 14:08:34.102522: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-04-17 14:08:34.102600: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-04-17 14:08:34.102606: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


env: CUBLAS_WORKSPACE_CONFIG=:16:8
cuda


In [5]:
print(compute_top_k_predictions_with_lang_filter)

<function compute_top_k_predictions_with_lang_filter at 0x7f8427d4cc10>


In [6]:
if IN_COLAB or IN_KAGGLE:
    if IN_COLAB:
        DATA_DIR_PATH = '/content/drive/MyDrive/ML/lecr/data/'
        LOCAL_MODEL_DIR = '/content/drive/MyDrive/ML/lecr/models/'
    elif IN_KAGGLE:
        DATA_DIR_PATH = '/kaggle/input/learning-equality-curriculum-recommendations/'
        LOCAL_MODEL_DIR = '/kaggle/input/lecr-one-trans-retreiver-ranker/'
    class CFG:
        FILENAME_DATA_CONTENT_CSV = DATA_DIR_PATH + "content.csv"
        FILENAME_DATA_TOPICS_CSV = DATA_DIR_PATH + "topics.csv"
        FILENAME_DATA_CORRELATIONS_CSV = DATA_DIR_PATH + "correlations.csv"
        FILENAME_DATA_SAMPLE_SUBMISSION_CSV = DATA_DIR_PATH + "sample_submission.csv"
        PRETRAINED_MODEL_DIR = LOCAL_MODEL_DIR
    if IN_KAGGLE:
        CFG.MINILM_PATH = '/kaggle/input/paraphrase-multilingual-minilm-l12-v2/paraphrase-multilingual-MiniLM-L12-v2'
        CFG.MPNET_PATH = '/kaggle/input/paraphrase-multilingual-mpnet-base-v2/paraphrase-multilingual-mpnet-base-v2'
        CFG.DISTILUSE_PATH = '/kaggle/input/distiluse-base-multilingual-cased-v2/distiluse-base-multilingual-cased-v2'
else:
    from cfg import *
    CFG.MINILM_PATH = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
    CFG.MPNET_PATH = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
    CFG.DISTILUSE_PATH = 'sentence-transformers/distiluse-base-multilingual-cased-v2'

CFG.retreiver_models = [[{'trans_name': CFG.MINILM_PATH,
                          'filename': 'One_trans_minilm_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_unique_id_batch_size_1100_epoch_20_recall_10_0.793.pth'}
                         ,
                         {'trans_name': CFG.MPNET_PATH,
                          'filename': 'one_trans_mpnet_ArcFace_0.4_focal_loss_2.0_fp16_ml64_train_shuffled_parent_title_new_sep_unique_id_batch_size_1050_epoch_20_recall_10_0.796.pth'}
                          ]
                        ,
                        [{'trans_name': CFG.DISTILUSE_PATH,
                          'filename': 'One_trans_distiluse_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_batch_size_1200_epoch_27_recall_10_0.787.pth'}]
                        ]

CFG.ranker_models = [[{'trans_name': CFG.MINILM_PATH,
                       'filename': 'One_trans_ranker_MiniLM_bce_ml64_batch_size_800_epoch_9_f2__0.636.pth',
                       'out_emb_dims': 384}
                      ,
                      {'trans_name': CFG.MPNET_PATH,
                       'filename': 'One_trans_ranker_mpnet_bce_ml64_batch_size_450_epoch_17_f2__0.657.pth',
                       'out_emb_dims': 768}
                      ]]

TEST_VAL_SUBSET = True
TESTING_SUBSET = False
DEBUG = False
if TEST_VAL_SUBSET or TESTING_SUBSET:
    DEBUG = True
K_VALS = [10]
INFERENCE_BATCH_SIZE = 2000 # 200 worked fine earlier. Experimenting with larger batch size
MAX_STRING_LENGTH = 1250

In [7]:
contents_df     = pd.read_csv(CFG.FILENAME_DATA_CONTENT_CSV)
topics_df       = pd.read_csv(CFG.FILENAME_DATA_TOPICS_CSV)
correlations_df = pd.read_csv(CFG.FILENAME_DATA_CORRELATIONS_CSV)
if TEST_VAL_SUBSET:
    VAL_CORR_IDS_F = CFG.DIRNAME_PROCESSED_DATA + "ranker_val_corr_ids.f"
    val_corr_ids_df = pd.read_feather(VAL_CORR_IDS_F)
    # sub_topic_ids = val_corr_ids_df[['t_id']].drop_duplicates()['t_id'].tolist()
    sub_df = pd.DataFrame(val_corr_ids_df.loc[val_corr_ids_df['label'] == 1].drop(['label'],axis=1).groupby('t_id')['c_id'].apply(list)).reset_index().rename(columns={'t_id':'topic_id', 'c_id':'content_ids'})
    correct_df = sub_df
else:
    sub_df = pd.read_csv(CFG.FILENAME_DATA_SAMPLE_SUBMISSION_CSV)
sub_topic_ids = sub_df['topic_id'].tolist()

In [8]:
if TESTING_SUBSET:
    SAMPLE_SIZE = 5000
    contents_df = contents_df.sample(n=SAMPLE_SIZE)

In [9]:
def limit_str(r):
    return (r[:MAX_STRING_LENGTH]) if pd.notnull(r) else r

#
# Limit the size of description and text in contents
#
contents_df['description'] = contents_df['description'].apply(lambda r: limit_str(r))
contents_df['text'] = contents_df['text'].apply(lambda r: limit_str(r))

In [10]:
# Pick all the topics in the subset file
# Retreiver:
# First two models share tokenizer, so we can organize the models by tokenizers
# Pick a tokenizer
# Do the FE + tokenizing for all the contents, for topics pick from the subset file
# Load the pre-trained models
# call compute_top_k_predictions for the topics
# Save the predictions
# Take a union of the predictions
# Create topic, content prediction pairs


In [11]:
class TransModel(torch.nn.Module):
    """
    Wrapper. Gets tokenized features and returns the output logits for whether the token and content are matching.
    """
    def __init__(self, trans_model_name):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(trans_model_name)
    
    def get_transformer_output(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids = input_ids, attention_mask=attention_mask)
        pooled_embeddings=mean_pooling_with_token_embs(outputs[0], attention_mask)
        return pooled_embeddings

    def forward(self, features):
        # Grab embeddings from transformer
        embs = self.get_transformer_output(
            features[:,0,:],
            features[:,1,:]
        )
        return embs


In [12]:
retreiver_preds = []
topics_subset_df = topics_df.query(f'id in {sub_topic_ids}').reset_index(drop=True)
for model_list in CFG.retreiver_models:
    auto_tokenizer = AutoTokenizer.from_pretrained(model_list[0]['trans_name'])
    pt('Tokenizing data ... ')
    topics_tokenized_df = fe_and_tokenize_df(auto_tokenizer, retreiver_topics_FE, post_process_df = None, text_cols = ['combined'], col_prefix = "t_", max_length = 64, debug = False, df = topics_df)

    contents_tokenized_df = fe_and_tokenize_df(auto_tokenizer, retreiver_contents_FE, post_process_df = None, text_cols = ['combined'], col_prefix = "c_", max_length = 64, debug = False, df = contents_df)

    # Pick topics that are only in correlations file
    topics_tokenized_df = topics_tokenized_df.query(f'id in {sub_topic_ids}').reset_index(drop=True)

    topics_tokenized_df   = topics_tokenized_df.reset_index(drop=True).reset_index()
    contents_tokenized_df = contents_tokenized_df.reset_index(drop=True).reset_index()

    t_cols = [col for col in topics_tokenized_df.columns if col.startswith('t_')]
    c_cols = [col for col in contents_tokenized_df.columns if col.startswith('c_')]
    features_data = (torch.tensor(np.array(topics_tokenized_df[t_cols].values.tolist())),
                     torch.tensor(np.array(contents_tokenized_df[c_cols].values.tolist()))
                     )

    for model in model_list:
        model_filename = CFG.PRETRAINED_MODEL_DIR + model['filename']
        pt(f'Loading model {model_filename} ... ')
        trans_model = TransModel(trans_model_name = model['trans_name'])
        load_model( {'trans_model': trans_model}, filename = model_filename)
        trans_model = trans_model.to(device)
        # preds = compute_top_k_predictions(trans_model,
        #                                   device,
        #                                   topics_features = features_data[0],
        #                                   contents_features = features_data[1],
        #                                   k_vals = K_VALS,
        #                                   batch_size = INFERENCE_BATCH_SIZE)
        preds = compute_top_k_predictions_with_lang_filter(trans_model,
                                          device,
                                          topics_features = features_data[0],
                                          contents_features = features_data[1],
                                          topics_df = topics_subset_df,
                                          contents_df = contents_df,
                                          k_vals = K_VALS,
                                          batch_size = INFERENCE_BATCH_SIZE,
                                          fill_value = -2
                                          )
        pt('Garbage collection ... ')
        del trans_model
        # report_gpu()
        retreiver_preds.append(preds)
        
## Take the union of all the predictions row-wise/topic-wise from the models for the same 'k' values
# The reason we use fill_val = -2, is because we take a rowise_intersection later to get the True-positives, and the other 2d array has -1 as fill value. So we don't want to have a false intersection with same fill values, so we use -2 as fill value here instead of -1.
matched_content_idxs_list = [np_rowise_unique(np.hstack([model_pred[i] for model_pred in retreiver_preds]), fill_value = -2) for i,_ in enumerate(K_VALS)]

2023-04-17 14:08:43.858395 : Tokenizing data ... 


Map (num_proc=2):   0% 0/61517 [00:00<?, ? examples/s]

Map (num_proc=2):   2% 1000/61517 [00:00<00:54, 1100.39 examples/s]

Map (num_proc=2):   5% 3000/61517 [00:01<00:17, 3346.01 examples/s]

Map (num_proc=2):   8% 5000/61517 [00:01<00:10, 5252.61 examples/s]

...

100% 6064/6070 [03:39<00:00, 27.09it/s]

100% 6067/6070 [03:39<00:00, 25.96it/s]

100% 6070/6070 [03:39<00:00, 25.12it/s]

100% 6070/6070 [03:39<00:00, 27.60it/s]

(6070, 10)
2023-04-17 14:33:08.345164 : Garbage collection ... 


In [13]:
if TEST_VAL_SUBSET:
    # Get mapping from contents id to idx
    contents_id_to_idx = {}
    for idx in range(0, len(contents_df)):
        contents_id_to_idx[contents_df.loc[idx]['id']] = idx

    correct_df_series = correct_df['content_ids'].apply(lambda id_list: [contents_id_to_idx[id] for id in id_list])
    
    recalls = calculate_mean_average_recall_at_k(matched_content_idxs_list, K_VALS, corr_data = correct_df_series, fill_value = -1)
    print(f'\nUnified Mean Average Recalls: {recalls}')


Unified Mean Average Recalls: {10: 0.841}


In [14]:
retreiver_preds_df = get_pred_corr_ids_df(topics_tokenized_df, contents_tokenized_df, matched_content_idxs_list[0])

In [15]:
if DEBUG:
    print(matched_content_idxs_list)

[array([[ 13201,  17332,     -2, ..., 140883, 141983, 150516],
       [  3333,     -2,  11776, ...,     -2,     -2, 141835],
       [ 18725,  24330,  31889, ..., 152670,     -2,     -2],
       ...,
       [  7350,  12069,  20216, ..., 142950,     -2, 145498],
       [  1509,     -2,     -2, ...,     -2, 129632,     -2],
       [  3135,     -2,     -2, ..., 150219,     -2,     -2]])]


In [16]:
if DEBUG:
    print(retreiver_preds_df)

                  t_id            c_id
0       t_00102869fbcb  c_15fafef749e7
1       t_2b13b4c94efc  c_15fafef749e7
2       t_4d2bd52d5898  c_15fafef749e7
3       t_67ee9da84a8d  c_15fafef749e7
4       t_7ada2538fb0f  c_15fafef749e7
...                ...             ...
113368  t_ffc71a181765  c_5942cae4dfcf
113369  t_ffc71a181765  c_713319180ec5
113370  t_ffc71a181765  c_a75bf1bc089f
113371  t_ffc71a181765  c_c817b6112048
113372  t_ffc71a181765  c_f0e4ca951269

[113373 rows x 2 columns]


In [17]:
if TEST_VAL_SUBSET:
    retreiver_preds_df.to_feather(CFG.DIRNAME_PROCESSED_DATA + "ranker_inference_pipeline_corr_ids.f")

In [18]:
# Ranker:
# Two models share tokenizer, so we again organize the models by tokenizers
# Pick a tokenizer
# Do the FE + tokenizing for the contents and the topics in the subset file
# take the topic, content prediction pairs and create the merged data
# For each of the topic, content pair get the prediction score
# Take the average of predictions
# Make the 1/0 prediction and only save the 1 predictions
# Collate topic, content list and write it out

In [19]:
class TransModel(torch.nn.Module):
    """
    Wrapper. Gets tokenized features and returns the output logits for whether the token and content are matching.
    """
    def __init__(self, trans_model_name, out_embs_size):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(trans_model_name)
        self.classifier = nn.Linear(out_embs_size, 1)
    
    def get_transformer_output(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids = input_ids, attention_mask=attention_mask)
        pooled_embeddings=mean_pooling_with_token_embs(outputs[0], attention_mask)
        return pooled_embeddings

    def forward(self, features):
        # Grab embeddings from transformer
        embs = self.get_transformer_output(
            features[:,0,:],
            features[:,1,:]
        )
        logits = self.classifier(embs)
        return logits


In [ ]:
ranker_preds = []
num_rankers = 0
for model_list in CFG.ranker_models:
    auto_tokenizer = AutoTokenizer.from_pretrained(model_list[0]['trans_name'])

    pt('Tokenizing data ... ')
    topics_tokenized_df, contents_tokenized_df = ranker_tokenize_main(auto_tokenizer, topics_df.copy(),  contents_df.copy(), max_length = 64)

    print('')

    pt(' Pick topics that are only in correlations file ... ')
    topics_tokenized_df = topics_tokenized_df.query(f'id in {sub_topic_ids}').reset_index(drop=True)

    topics_tokenized_df   = topics_tokenized_df.reset_index(drop=True)
    contents_tokenized_df = contents_tokenized_df.reset_index(drop=True)

    merged_df = get_ranker_topic_content_merged_data(auto_tokenizer, retreiver_preds_df, topics_tokenized_df, contents_tokenized_df, keep_ids = True)

    feature_cols = ['input_ids', 'attention_mask']
    merged_features_data = torch.tensor(np.array(merged_df[feature_cols].values.tolist()))
    merged_df = merged_df.drop(feature_cols,axis=1)

    dl = DataLoader(merged_features_data, batch_size=INFERENCE_BATCH_SIZE, shuffle=False, num_workers = 2)

    for model in model_list:
        num_rankers += 1
        model_filename = CFG.PRETRAINED_MODEL_DIR + model['filename']
        pt(f'Loading model {model_filename} ... ')
        trans_model = TransModel(trans_model_name = model['trans_name'], out_embs_size = model['out_emb_dims'])
        load_model( {'trans_model': trans_model}, filename = model_filename)
        trans_model = trans_model.to(device)
        trans_model.eval()
        model_probs = []
        with torch.no_grad():
            for features_data in tqdm(dl, desc = 'Inference'):
                logits = trans_model(features_data.to(device))
                probs = torch.sigmoid(logits)
                model_probs.append(probs)
        model_probs = torch.concat(model_probs, axis = 0)
        pt('Garbage collection ... ')
        del trans_model
        # report_gpu()
        merged_df['label'] = model_probs.cpu()
        ranker_preds.append(merged_df)
# merged_preds = torch.round(torch.sum(torch.concat(ranker_probs, axis = 1), dim = 1)/num_rankers)

In [21]:
#ranker_preds_df = ranker_preds[0].copy()
ranker_preds_df = pd.concat(ranker_preds).groupby(['t_id', 'c_id']).sum().reset_index()
ranker_preds_df['label'] = ranker_preds_df['label'].apply(lambda v: round(v/num_rankers))

In [22]:
if DEBUG:
    print(ranker_preds_df)

In [23]:
# Attach the predictions to t_id, c_id pairs
# ranker_preds_df['label'] = merged_preds.cpu().tolist()

# Pick only the pairs predicted to be correlated
correlated_pairs_df = ranker_preds_df.loc[ranker_preds_df['label'] == 1.0].drop(['label'], axis = 1)

# Convert pairs to topic_id-> content_ids list
correlated_preds_df = correlated_pairs_df.groupby(['t_id']).agg({'c_id': lambda x: " ".join(x)}).reset_index()

correlated_preds_df = correlated_preds_df.rename(columns = {'t_id':'topic_id', 'c_id':'content_ids'})

# merge sub_df and correlated_preds_df to set the ordering 
correlated_preds_df = pd.merge(sub_df[["topic_id"]], correlated_preds_df, how="left", on="topic_id")

correlated_preds_df = correlated_preds_df.fillna(value={"content_ids":""})


In [24]:
if DEBUG:
    print(correlated_preds_df)

In [25]:
if TEST_VAL_SUBSET:
    print(correct_df)
    preds_df = correlated_preds_df.rename(columns = {'topic_id':'t_id', 'content_ids':'c_id'})
    correct_df_ = correct_df.rename(columns = {'topic_id':'t_id', 'content_ids':'c_id'})
    preds_df['c_id'] = preds_df['c_id'].apply(lambda r: r.split())
    correct_np = get_np_from_retreival_df(correct_df_, fill_value = '-1')
    pred_np    = get_np_from_retreival_df(preds_df, fill_value = '-2')
    print(get_avg_recall_precision_f2_from_np(correct_np, pred_np))
else:
    correlated_preds_df.to_csv('submission.csv', index = None)